# 学習結果の可視化

`pso_lstm_5m.ipynb` で学習後、以下を表示します。
- **学習曲線**: `final_log_pso_lstm_5m.csv` の Epoch と Loss（train / val）
- **実測 vs 予測**: テストデータについて、終値（価格スケール）の時系列

**前提**: メインノートブックで `final_log_pso_lstm_5m.csv` が生成されていること。
実測 vs 予測は、メインノートで `test_result.npz` を保存している場合に表示可能。

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pso_lstm_common import compute_metrics

try:
    import japanize_matplotlib
except ImportError:
    plt.rcParams["font.family"] = ["DejaVu Sans", "Yu Gothic", "Meiryo"]

## 1. 学習曲線（Epoch vs Loss）

In [ ]:
log_path = "final_log_pso_lstm_5m.csv"
if os.path.isfile(log_path):
    log_df = pd.read_csv(log_path)
    fig, ax = plt.subplots(figsize=(8, 4))
    if "loss" in log_df.columns:
        ax.plot(log_df["epoch"], log_df["loss"], label="Train Loss")
    if "val_loss" in log_df.columns:
        ax.plot(log_df["epoch"], log_df["val_loss"], label="Val Loss")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss (MSE)")
    ax.legend()
    ax.set_title("学習曲線")
    plt.tight_layout()
    plt.show()
else:
    print(f"{log_path} が見つかりません。先に pso_lstm_5m.ipynb を実行してください。")

## 2. 実測 vs 予測（終値・価格スケール）

In [ ]:
npz_path = "test_result.npz"
if os.path.isfile(npz_path):
    data = np.load(npz_path)
    y_test = data["y_test"]
    y_pred = data["y_pred"]
    lc_test = data["lc_test"]

    # 評価指標（対数収益スケール）
    rmse, mae, mape, r2 = compute_metrics(y_test, y_pred)
    corr = np.corrcoef(y_test.ravel(), y_pred.ravel())[0, 1] if y_test.size > 1 else 0.0
    dir_acc = (np.sign(y_test.ravel()) == np.sign(y_pred.ravel())).mean() * 100.0

    print("=== テストデータ評価指標（対数収益スケール） ===")
    print("【主な指標】")
    print(f"  相関係数:     {corr:.6f}")
    print(f"  方向一致率:   {dir_acc:.2f}%")
    print("【補助指標】")
    print(f"  RMSE:  {rmse:.6f}  MAE: {mae:.6f}")
    print(f"  MAPE:  {mape:.2f}%  R2: {r2:.6f}")

    actual_close = lc_test * np.exp(y_test)
    pred_close = lc_test * np.exp(y_pred)
    n = len(actual_close)
    x_axis = np.arange(n)
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(x_axis, actual_close, label="実測終値", alpha=0.8)
    ax.plot(x_axis, pred_close, label="予測終値", alpha=0.8)
    ax.set_xlabel("テストサンプル")
    ax.set_ylabel("終値")
    ax.legend()
    ax.set_title("テストデータ: 実測 vs 予測（終値）")
    plt.tight_layout()
    plt.show()
else:
    print(f"{npz_path} が見つかりません。実測 vs 予測を保存するには、pso_lstm_5m.ipynb で np.savez の行を有効にしてください。")